In [ ]:
# %% Import required packages
import anfis
from pyfume.Clustering import Clusterer
from pyfume.EstimateAntecendentSet import AntecedentEstimator
from pyfume.EstimateConsequentParameters import ConsequentEstimator
from pyfume.SimpfulModelBuilder import SugenoFISBuilder
from pyfume.Tester import SugenoFISTester
from pandas import read_csv
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, explained_variance_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [ ]:
# %% Train initial TS model and get performance metrics
cl = Clusterer(x_train=X_train, y_train=y_train, nr_clus=10)
clust_centers, part_matrix, _ = cl.cluster(method='fcm')
ae = AntecedentEstimator(X_train, part_matrix)
antecedent_params = ae.determineMF()
ce = ConsequentEstimator(X_train, y_train, part_matrix)
conseq_params = ce.suglms()
modbuilder = SugenoFISBuilder(antecedent_params, conseq_params, var_names, save_simpful_code=False)
model = modbuilder.get_model()
modtester = SugenoFISTester(model, X_test, var_names)
y_pred = modtester.predict()[0]
mse_init = mean_squared_error(y_test, y_pred)
print("Mean Squared Error: {:.3f}".format(mse_init))
mape_init = mean_absolute_percentage_error(y_test, y_pred)
print("Mean Absolute Percentage Error: {:.1f}%".format(mape_init*100))
exp_var_init = explained_variance_score(y_test, y_pred)
print("Explained Variance Score: {:.3f}".format(exp_var_init))

# %% Initialize ANFIS using the trained TS model 
anf = anfis.ANFIS(X_train, y_train, model, conseq_params)

# %% Optimize ANFIS model and get performance metrics
anf.trainHybridJangOffLine(epochs=20)
y_pred = anfis.predict(anf, X_test)
mse_final = mean_squared_error(y_test, y_pred)
print("Mean Squared Error: {:.3f}".format(mse_final))
mape_final = mean_absolute_percentage_error(y_test, y_pred)
print("Mean Absolute Percentage Error: {:.1f}%".format(mape_final*100))
exp_var_final = explained_variance_score(y_test, y_pred)
print("Explained Variance Score: {:.3f}".format(exp_var_final))